In [1]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

# Read in the CSV files
df = pd.read_csv('Batting.csv')

# Print first few rows of `batting_df`
df.head()

# Create Batting Average (`AVE`) column
df['AVE'] = df['H'] / df['AB']

# Create On Base Percent (`OBP`) column
plate_appearances = (df['AB'] + df['BB'] + df['SF'] + df['SH'] + df['HBP'])
df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / plate_appearances

# Create Slugging Percent (`Slug_Percent`) column
df['single'] = ((df['H'] - df['2B']) - df['3B']) - df['HR']
single = ((df['H'] - df['2B']) - df['3B']) - df['HR']
df['Slug_Percent'] = ((df['HR'] * 4) + (df['3B'] * 3) + (df['2B'] * 2) + single) / df['AB']

# Create On Base plus Slugging Percent (`OPS`) column
hr = df['HR'] * 4
triple = df['3B'] * 3
double = df['2B'] * 2
df['OPS'] = df['OBP'] + df['Slug_Percent']

#create fantasy value
df['fantasy_value'] = df['single'] + (df['2B'] * 2) + (df['3B'] * 3) + (df['HR'] *4) + df['R'] + df['RBI'] + df['BB'] + (df['SB'] * 2) - df['CS']


# Retrieve `df` information
df.info()

# Retrieve `df` columns
print(df.columns)

# Check for null values
print(df.isnull().sum(axis=0).tolist())

# Eliminate rows with null values
df = df.dropna()

print("filter df for players up to 2017")
# Filter `df` for players so only up to 2017
data = df.loc[df['yearID'] <= 2015]
print(data.tail())
#print(df_hitters.head())

print("filter df for only 2018 players")
# Filter `df` for year 2018 to use on test data
test_data = df.loc[df['yearID'] == 2016]
test_data.info()
print(test_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102816 entries, 0 to 102815
Data columns (total 28 columns):
playerID         102816 non-null object
yearID           102816 non-null int64
stint            102816 non-null int64
teamID           102816 non-null object
lgID             102079 non-null object
G                102816 non-null int64
AB               102816 non-null int64
R                102816 non-null int64
H                102816 non-null int64
2B               102816 non-null int64
3B               102816 non-null int64
HR               102816 non-null int64
RBI              102392 non-null float64
SB               101516 non-null float64
CS               79360 non-null float64
BB               102816 non-null int64
SO               94978 non-null float64
IBB              66251 non-null float64
HBP              100006 non-null float64
SH               96478 non-null float64
SF               66782 non-null float64
GIDP             76706 non-null float64
AVE             

In [2]:
def preprocess_features(master_df):
  """Prepares input features from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including
    synthetic features.
  """
  selected_features = master_df[['AVE', 'OBP', 'Slug_Percent', 'OPS', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB']]

  processed_features = selected_features.copy()
  # Create a synthetic feature.

  return processed_features

def preprocess_targets(lahmans_baseball_dataframe):
  """Prepares target features (i.e., labels) from California housing data set.

  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data
      from the California housing data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  # Scale the target to be in units of thousands of dollars.
  output_targets["fantasy_value"] = lahmans_baseball_dataframe["fantasy_value"]

  return output_targets


# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(data.head(30000))
training_targets = preprocess_targets(data.head(30000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(data.tail(15000))
validation_targets = preprocess_targets(data.tail(15000))

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])


def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a neural net regression model.

    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """

    # Convert pandas data into a dict of np arrays.
    features = {key: np.array(value) for key, value in dict(features).items()}

    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features, targets))  # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)

    # Shuffle the data, if specified.
    if shuffle:
        ds = ds.shuffle(10000)

    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels


def train_nn_regression_model(
        learning_rate,
        steps,
        batch_size,
        hidden_units,
        training_examples,
        training_targets,
        validation_examples,
        validation_targets):
    """Trains a neural network regression model.

    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.

    Args:
      learning_rate: A `float`, the learning rate.
      steps: A non-zero `int`, the total number of training steps. A training step
        consists of a forward and backward pass using a single batch.
      batch_size: A non-zero `int`, the batch size.
      hidden_units: A `list` of int values, specifying the number of neurons in each layer.
      training_examples: A `DataFrame` containing one or more columns from
        `california_housing_dataframe` to use as input features for training.
      training_targets: A `DataFrame` containing exactly one column from
        `california_housing_dataframe` to use as target for training.
      validation_examples: A `DataFrame` containing one or more columns from
        `california_housing_dataframe` to use as input features for validation.
      validation_targets: A `DataFrame` containing exactly one column from
        `california_housing_dataframe` to use as target for validation.

    Returns:
      A `DNNRegressor` object trained on the training data.
    """

    periods = 10
    steps_per_period = steps / periods

    # Create a DNNRegressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    dnn_regressor = tf.estimator.DNNRegressor(
        feature_columns=construct_feature_columns(training_examples),
        hidden_units=hidden_units,
        optimizer=my_optimizer,
    )

    # Create input functions.
    training_input_fn = lambda: my_input_fn(training_examples,
                                            training_targets["fantasy_value"],
                                            batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples,
                                                    training_targets["fantasy_value"],
                                                    num_epochs=1,
                                                    shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples,
                                                      validation_targets["fantasy_value"],
                                                      num_epochs=1,
                                                      shuffle=False)

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("RMSE (on training data):")
    training_rmse = []
    validation_rmse = []
    for period in range(0, periods):
        # Train the model, starting from the prior state.
        dnn_regressor.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        # Take a break and compute predictions.
        training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([item['predictions'][0] for item in training_predictions])

        validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])

        # Compute training and validation loss.
        training_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(training_predictions, training_targets))
        validation_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(validation_predictions, validation_targets))
        # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
        # Add the loss metrics from this period to our list.
        training_rmse.append(training_root_mean_squared_error)
        validation_rmse.append(validation_root_mean_squared_error)
    print("Model training finished.")

    # Output a graph of loss metrics over periods.
    plt.ylabel("RMSE")
    plt.xlabel("Periods")
    plt.title("Root Mean Squared Error vs. Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label="training")
    plt.plot(validation_rmse, label="validation")
    plt.legend()

    print("Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error)
    print("Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error)

    return dnn_regressor

dnn_regressor = train_nn_regression_model(
    learning_rate=0.0005,
    steps=5000,
    batch_size=100,
    hidden_units=[10, 5],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

Training examples summary:


,AVE,OBP,Slug_Percent,OPS,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.204376,0.256213,0.290105,0.546318,61.194067,168.959933,21.096767,43.324200,7.160933,1.152667,4.040567,19.736000,3.039600,1.597067,16.235000
std,0.118846,0.130398,0.186198,0.302934,48.996461,190.909718,27.588398,53.635095,9.509335,2.077876,7.227443,26.781173,7.709467,3.029381,22.026188
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.142857,0.185185,0.171429,0.364389,20.000000,16.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,0.227273,0.285714,0.312500,0.600074,44.000000,74.000000,7.000000,15.000000,2.000000,0.000000,1.000000,6.000000,0.000000,0.000000,5.000000
75%,0.268412,0.333333,0.399319,0.727432,101.000000,296.000000,34.000000,74.000000,12.000000,1.000000,5.000000,31.000000,2.000000,2.000000,25.000000
max,1.000000,1.000000,4.000000,5.000000,165.000000,705.000000,146.000000,240.000000,54.000000,21.000000,61.000000,153.000000,130.000000,42.000000,151.000000


Validation examples summary:


,AVE,OBP,Slug_Percent,OPS,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,0.197646,0.244526,0.295012,0.539538,58.913133,164.015600,21.687733,42.742733,8.544133,0.88960,4.881200,20.648667,2.789600,1.108467,15.283533
std,0.133097,0.146808,0.211074,0.344839,48.609748,193.254249,28.970314,54.527297,11.430991,1.78113,8.218767,28.619755,6.592822,2.211205,21.773493
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.117647,0.153846,0.140351,0.302987,19.000000,9.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.227848,0.285714,0.333333,0.622382,39.000000,64.000000,6.000000,13.000000,2.000000,0.00000,1.000000,6.000000,0.000000,0.000000,4.000000
75%,0.271429,0.333333,0.422541,0.752675,96.000000,284.000000,35.000000,72.000000,14.000000,1.00000,6.000000,33.000000,2.000000,1.000000,23.000000
max,1.000000,1.000000,4.000000,5.000000,163.000000,716.000000,146.000000,262.000000,56.000000,23.00000,64.000000,160.000000,78.000000,24.000000,232.000000


Training targets summary:


,fantasy_value
count,30000.000000
mean,126.462067
std,163.010840
min,-1.000000
25%,6.000000
50%,40.000000
75%,210.000000
max,789.000000


Validation targets summary:


,fantasy_value
count,15000.000000
mean,129.800333
std,171.738655
min,-1.000000
25%,3.000000
50%,38.000000
75%,214.000000
max,845.000000


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_save_checkpoints_steps': None, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_master': '', '_evaluation_master': '', '_train_distribute': None, '_is_chief': True, '_log_step_count_steps': 100, '_session_config': None, '_task_id': 0, '_model_dir': '/tmp/tmpr0ymgysy', '_num_ps_replicas': 0, '_save_summary_steps': 100, '_service': None, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa2b8a95358>}
Training model...
RMSE (on training data):
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpr0ymgy

  period 04 : 9.47
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpr0ymgysy/model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2501 into /tmp/tmpr0ymgysy/model.ckpt.
INFO:tensorflow:step = 2501, loss = 13427.413
INFO:tensorflow:global_step/sec: 699.916
INFO:tensorflow:step = 2601, loss = 6868.7847 (0.143 sec)
INFO:tensorflow:global_step/sec: 1047.07
INFO:tensorflow:step = 2701, loss = 8809.377 (0.096 sec)
INFO:tensorflow:global_step/sec: 1020.58
INFO:tensorflow:step = 2801, loss = 3168.9194 (0.098 sec)
INFO:tensorflow:global_step/sec: 360.053
INFO:tensorflow:step = 2901, loss = 6924.218 (0.280 sec)
INFO:tensorflow:Saving checkpoints for 3000 into /tmp/tmpr0ymgysy/model.ckpt.
INFO:tensorflow:Loss for final step: 6402.9077.
INFO:tensorflow:Calling m

In [3]:
test_examples = preprocess_features(test_data)
test_targets = preprocess_targets(test_data)

predict_testing_input_fn = lambda: my_input_fn(test_examples,
                                               test_targets["fantasy_value"],
                                               num_epochs=1,
                                               shuffle=False)

test_predictions = dnn_regressor.predict(input_fn=predict_testing_input_fn)


test_predictions = np.array([item['predictions'][0] for item in test_predictions])

final_dict = {}
for i in range(20):
    print(test_predictions[i])
    print(test_data.iloc[i])
    final_dict[i] = test_predictions[i]


root_mean_squared_error = math.sqrt(
    metrics.mean_squared_error(test_predictions, test_targets))

print("Final RMSE (on test data): %0.2f" % root_mean_squared_error)

import operator

sorted_dict = sorted(final_dict.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_dict)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpr0ymgysy/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1.9119946
playerID         abadfe01
yearID               2016
stint                   1
teamID                MIN
lgID                   AL
G                      39
AB                      1
R                       0
H                       0
2B                      0
3B                      0
HR                      0
RBI                     0
SB                      0
CS                      0
BB                      0
SO                      1
IBB                     0
HBP                     0
SH                      0
SF                      0
GIDP                    0
AVE                     0
OBP                     0
single                  0
Slug_Percent            0
OPS                     0
fantasy_value       